# Запуск Spark на Yarn

In [ ]:
from pyspark.sql import SparkSession

Apache Spark может быть запущен на Yarn. Воркеры будут запущены на узлах HDFS ближе к данных, что позволит достичь максимальной производительности в силу минимизации передачи данных по сети.

Для запуска Apache Spark на Yarn необходимо:

1. Выбрать [версию](https://spark.apache.org/downloads.html) Apache Spark, которая была собрана для вашей версии Hadoop;
1. Указать расположение конфигов Hadoop при помощи переменных окружения: `HADOOP_CONF_DIR` и `YARN_CONF_DIR`;
1. Указать `--master yarn` при старте приложения через `spark-submit` или [`master("yarn")`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.builder.master.html) при старте через PySpark

Spark по умолчанию запросит три контейнера:

- один контейнер для Application Master (драйвер),
- два контейнера для Воркеров.

Число контейненров для воркеров можно настраивать через `spark.executor.instances`.

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Yarn")
        .master("yarn")
        .config("spark.executor.instances", 3)
        .getOrCreate()
)
sc = spark.sparkContext

In [ ]:
df = spark.sql("select 'Hello, Yarn!' as message")

In [ ]:
df.show()

# Проверка доступа к Hadoop

Hadoop стартует в однонодовом режиме: все демоны на одном компьютере. Демоны Hadoop:

- демон [NameNode](http://localhost:9870) запущен на `9870` порту,
- демон [DataNode](http://localhost:9864) запущен на `9864` порту,
- демон [ResourceManager](http://localhost:8088) запущен на `8088` порту порту,
- демон [NodeManager](http://localhost:8042) запущен на `8042` порту,

Для удобства текущий ноутбук сконфигурирован таким образом, что можно напрямую вызывать команды для работы с HDFS и Yarn:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -ls /

In [ ]:
! source ~/.bash_aliases && \
yarn application -list

In [ ]:
spark.stop()

In [ ]:
! source ~/.bash_aliases && \
yarn application -list

## Конфигурация

Для запуска Apache Spark в Yarn необходимо положить `hdfs-site.xml` и `yarn-site.xml` в директории `$HADOOP_CONF_DIR` и `$YARN_CONF_DIR` соответственно:

In [ ]:
! cat $HADOOP_CONF_DIR/hdfs-site.xml

In [ ]:
! cat $YARN_CONF_DIR/yarn-site.xml

В то же время обе переменные могут указывать на одну и ту же директорию:

In [ ]:
! echo "HADOOP_CONF_DIR: $HADOOP_CONF_DIR"

In [ ]:
! echo "YARN_CONF_DIR: $YARN_CONF_DIR"

## Чтение и запись данных

In [ ]:
import os
namenode = os.environ['NAMENODE_HOST']

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Yarn")
        .master("yarn")
        .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse")
        .getOrCreate()
)
sc = spark.sparkContext

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -mkdir -p /user/jovyan

In [ ]:
(
spark.range(0, 1000, 1, 10)
    .write
    .mode("overwrite")
    .save(f"hdfs://{namenode}:9000/user/jovyan/range")
)

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /user/jovyan/range

In [ ]:
(
spark.read
    .parquet(f"hdfs://{namenode}:9000/user/jovyan/range")
    .count()
)

После запуска в Yarn приложение Spark будет по умолчанию будет искать данные в HDFS.

Можно указывать как полный путь:

In [ ]:
(
spark.read
    .parquet("/user/jovyan/range")
    .count()
)

Также можно указать путь относительно домашней директории:

In [ ]:
(
spark.read
    .parquet("range")
    .count()
)

## Spark Warehouse

При этом Spark Warehouse также окажется в HDFS:

In [ ]:
(
spark.range(0, 1000, 1, 10)
    .write
    .mode("overwrite")
    .saveAsTable("range_table")
)

In [ ]:
spark.sql("select count(*) from range_table").show()

Spark автоматически развернул путь до Spark Warehouse в полную ссылку:

In [ ]:
print(f"Spark Warehouse Location: {spark.conf.get('spark.sql.warehouse.dir')}")

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /tmp/spark-warehouse

**Примечание:** `hdfs://hadoop:9000` - это адрес NameNode, который указывается в параметре `fs.defaultFS` через `core-site.xml`:

In [ ]:
! cat $HADOOP_CONF_DIR/core-site.xml

## Spark Catalog

Spark Catalog содержит список созданных таблиц, UDF функций и пр.:

In [ ]:
spark.catalog.listTables()

Spark Catalog желательно сохранять в какое-нибудь постоянное хранилище, т.к. после перезапуска приложения:

In [ ]:
spark.stop()

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Yarn")
        .master("yarn")
        .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse")
        .getOrCreate()
)
sc = spark.sparkContext

все данные из каталога пропадут, а значит нельзя будет выполнять запросы к созданным таблицам:

In [ ]:
import sys
try:
    spark.sql("select count(*) from range_table").show()
except Exception as e:
    print(e, file=sys.stderr)

Spark Catalog пуст:

In [ ]:
spark.catalog.listTables()

При этом файлы таблицы `range_table` по прежнему находятся в HDFS:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -find /tmp/spark-warehouse

Таким образом, можно заключить, что Spark Catalog по умолчанию демонстрирует признаки временного хранилища.

Все так и есть: если никак не конфигурировать Spark Catalog, то каталог будет храниться в памяти и исчезать, после того, как приложение завершает свою работу

> **Spark Catalog по умолчанию хранится в памяти!**

### Постоянное хранение Spark Catalog

Самая простая настройка - это использование метода [`SparkSession.builder#enableHiveSupport`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.builder.enableHiveSupport.html) во время создания сессии:

In [ ]:
spark.stop()

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Yarn")
        .master("yarn")
        .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse")
        .enableHiveSupport()
        .getOrCreate()
)
sc = spark.sparkContext

Если никак больше не конфигурировать Spark, то Spark Catalog будет сохраняться во встраиваемую базу данных [Apache Derby](https://db.apache.org/derby/).

Теперь если создать новую таблицу:

In [ ]:
(
spark.range(0, 1000, 1, 10)
    .write
    .mode("overwrite")
    .saveAsTable("range_table_with_hive_support")
)

In [ ]:
spark.sql("select count(*) from range_table_with_hive_support").show()

То в текущей директории появится 2 объекта:

- `derby.log` - логи работы встраиваемой базы данных Derby,
- `metastore_db` - файлы базы данных Derby, в которой сохраняется Spark Catalog.

In [ ]:
! ls -l

Каталог будет храниться в базе данных, и между перезапусками приложения можно будет обращаться ко всем объектам:

In [ ]:
spark.stop()

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Yarn")
        .master("yarn")
        .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse")
        .enableHiveSupport()
        .getOrCreate()
)
sc = spark.sparkContext

In [ ]:
spark.sql("select count(*) from range_table_with_hive_support").show()

Запрос отработал успешно, т.к. каталог в текущем Spark приложении непустой:

In [ ]:
spark.catalog.listTables()

Локальное расположение Derby невозможно изменить, поэтому файл derby.log и директория `metastore_db` всегда будет сохраняться в текущей директории.

> **Невозможно изменить место хранения Derby на локальном жестком диске**

### Надежное хранение Spark Catalog в Hive MetaStore

Дополнительно можно добавить файл `hive-site.xml` в `$HADOOP_CONF_DIR`, в котором описаны параметры Hive, и тогда Spark вместо хранения данных в локальной Derby базе будет хранить каталог в Hive MetaStore.

Кроме `enableHiveSupport()` ничего другого не потребуется.

В текущем окружении отсутствует Hive, поэтому нет возможности продемонстирировать этот пример на практике.

In [ ]:
spark.stop()

## Выводы

Поддержка планировщика Yarn появилась в Apache Spark в версии `0.6`, и с того времени Yarn получил наибольшее внимание от разработчиков Apache Spark, что делает его самым надежным способом запуска задач Spark на кластере. В то же время Yarn является одним из двух основных компонентов поставки Hadoop (второй - это HDFS), и поэтому он очень тесно интегрирован с HDFS. Такая тесая связь очень сильно повышает уровень локальности данных: задачи запускаются на тех машинах, на которых хранятся данные, реализуя концепцию перемешения кода поближе к данным.

Задачи Spark, запущенные на Yarn, значительно превосходят по скорости исполнения другие планировщики, а поэтому проекты, которым нужна надежность и скорость еще долгое время будут выбирать связку Spark + Yarn.

## Задание

1. Сохранить в HDFS последовательность из десяти чисел Фиббоначчи:
    - в виде таблицы
    - в формате json
    - с двумя колонками:
        - номер числа,
        - значение числа фиббоначчи по этому номеру.
1. Убедиться, что таблица остается доступной между перезапусками.
1. Что произойдет, если удалить `derby.log`? Провести эксперименты.
1. Что произойдет, если удалить `metastore_db`? Провести эксперименты.

### Ответы

1. Сохранить в HDFS последовательность из десяти квадратов чисел:
    - в виде таблицы
    - в формате json
    - в одной партиции
    - с двумя колонками:
        - значение числа,
        - квадрат числа.

<details>
    <summary>Ответ</summary>

1. Сохранить файл:
```python
from pyspark.sql import functions as F

# запустить приложение
spark = (
    SparkSession
        .builder
        .appName("Yarn")
        .master("yarn")
        .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse")
        .enableHiveSupport()
        .getOrCreate()
)

# сгенерировать датафрейм
df = spark.range(0, 10).withColumn("square", F.expr("id * id"))
# сохранить как таблицу
df.repartition(1).write.mode("overwrite").format("json").saveAsTable("squares")

```

2. Проверить файлы в HDFS:
```bash
! source ~/.bash_aliases && \
hdfs dfs -cat /tmp/spark-warehouse/squares/*.json
```

</details>

2. Убедиться, что таблица остается доступной между перезапусками.

<details>
    <summary>Ответ</summary>

```python
# остановить приложение
spark.stop()

# запустить приложение снова
spark = (
    SparkSession
        .builder
        .appName("Yarn")
        .master("yarn")
        .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse")
        .enableHiveSupport()
        .getOrCreate()
)

# таблица доступна для запросов
spark.sql("select * from squares").show()
# таблица находится в каталоге
spark.catalog.listTables()
```

</details>

3. Что произойдет, если удалить `derby.log`?

<details>
    <summary>Ответ</summary>

Ошибок не будет, приложение продолжит ответчать на запросы к таблице.

Для экспериментов может потребоваться перезапуск kernel через главное меню:

    Kernel -> Restart Kernel -> Restart

</details>

4. Что произойдет, если удалить `metastore_db`?

<details>
    <summary>Ответ</summary>

Работоспособность запросов нарушится в текущей сессии, но таблицы будут недоступны для запросов после перезапуска.

Для экспериментов может потребоваться перезапуск kernel через главное меню:

    Kernel -> Restart Kernel -> Restart

</details>